In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score

from xgboost import XGBClassifier

import itertools

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import practice_code as pc
import individual_stats as ind

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
clutch_time = pd.read_csv('CSV Files/clutch_time.csv')

In [3]:
reg = pd.read_csv('CSV Files/regular_season.csv')

In [4]:
train_target= clutch_time[clutch_time['data_set'] == '2015-2016 Regular Season']

In [5]:
train_clutch_time = clutch_time[clutch_time['data_set'] != '2015-2016 Regular Season']

In [6]:
ids = ind.add_ids(reg)

In [7]:
clutch_time = pc.replace_nan(clutch_time)
clutch_time['total_blocks'] = (clutch_time['block'] != '')*1
clutch_time['assist_count'] = (clutch_time['assist'] != '')*1

In [8]:
player_stats_clutch = ind.object_to_int(clutch_time, 'points')
assists = ind.assist_stats(player_stats_clutch)
blks = ind.blk_stats(player_stats_clutch)
names = ind.player_names(player_stats_clutch)
train_clutch_stats = ind.merge_all(names, blks, assists, player_stats_clutch)
train_clutch_stats = ind.get_per_game_stats(train_clutch_stats)
train_clutch_stats = ind.merge_ids(train_clutch_stats, ids)

/home/ubuntu/clutch-baller/individual_stats.py:10: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  new_df = df.convert_objects(convert_numeric=True)


In [9]:
player_stats_train = ind.object_to_int(train_target, 'points')
assists = ind.assist_stats(player_stats_train)
blks = ind.blk_stats(player_stats_train)
names = ind.player_names(player_stats_train)
train_target_stats = ind.merge_all(names, blks, assists, player_stats_train)
train_target_stats = ind.get_per_game_stats(train_target_stats)
train_target_stats = ind.merge_ids(train_target_stats, ids)

/home/ubuntu/clutch-baller/individual_stats.py:10: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  new_df = df.convert_objects(convert_numeric=True)


In [10]:
train_clutch_stats = ind.add_league_avg(train_clutch_stats)
train_clutch_stats = ind.is_clutch(train_clutch_stats)

/home/ubuntu/clutch-baller/individual_stats.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['clutch_all_shots'] = (df['shooting%'] > df['league_all_shot_avg']) & (df['total_shots']>50)
/home/ubuntu/clutch-baller/individual_stats.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['clutch_2pt'] = (df['2pt%'] > df['league_2pt_avg']) & (df['2fg_attempts']>35)
/home/ubuntu/clutch-baller/individual_stats.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [11]:
train_target_stats = ind.add_league_avg(train_target_stats)


In [12]:
train_target_stats = ind.clutch_for_season(train_target_stats)

/home/ubuntu/clutch-baller/individual_stats.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['clutch_all_shots'] = (df['shooting%'] > df['league_all_shot_avg']) & (df['total_shots']>10)
/home/ubuntu/clutch-baller/individual_stats.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['clutch_2pt'] = (df['2pt%'] > df['league_2pt_avg'])
/home/ubuntu/clutch-baller/individual_stats.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [13]:
test_target = pd.read_csv('Test_data/first_test.csv')

In [14]:
test_target = test_target.drop('id', axis =1)

In [15]:
#test_target = ind.merge_ids(test_target, ids)

In [16]:
test_clutch_time = pc.replace_nan(clutch_time)
test_clutch_time['total_blocks'] = (test_clutch_time['block'] != '')*1
test_clutch_time['assist_count'] = (test_clutch_time['assist'] != '')*1

In [17]:
player_stats_test = ind.object_to_int(test_clutch_time, 'points')
assists = ind.assist_stats(player_stats_test)
blks = ind.blk_stats(player_stats_test)
names = ind.player_names(player_stats_test)
test_clutch_stats = ind.merge_all(names, blks, assists, player_stats_test)
test_clutch_stats = ind.get_per_game_stats(test_clutch_stats)
test_clutch_stats = ind.merge_ids(test_clutch_stats, ids)

/home/ubuntu/clutch-baller/individual_stats.py:10: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  new_df = df.convert_objects(convert_numeric=True)


In [18]:
test_clutch_stats = ind.add_league_avg(test_clutch_stats)
test_clutch_stats = ind.is_clutch(test_clutch_stats)

/home/ubuntu/clutch-baller/individual_stats.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['clutch_all_shots'] = (df['shooting%'] > df['league_all_shot_avg']) & (df['total_shots']>50)
/home/ubuntu/clutch-baller/individual_stats.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['clutch_2pt'] = (df['2pt%'] > df['league_2pt_avg']) & (df['2fg_attempts']>35)
/home/ubuntu/clutch-baller/individual_stats.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [19]:

test_target = ind.merge_ids(test_target, ids)

In [20]:
test_target = ind.add_league_avg(test_target)

In [21]:
test_target = ind.clutch_for_season(test_target)

/home/ubuntu/clutch-baller/individual_stats.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['clutch_all_shots'] = (df['shooting%'] > df['league_all_shot_avg']) & (df['total_shots']>10)
/home/ubuntu/clutch-baller/individual_stats.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['clutch_2pt'] = (df['2pt%'] > df['league_2pt_avg'])
/home/ubuntu/clutch-baller/individual_stats.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [22]:
new_id = []
v = test_target['id'].max()
for value in test_target['id']:
    if value == 0:
        v +=1
        new_id.append(v)
    else:
        new_id.append(value)
test_target['id'] = new_id

In [23]:
# making sure that i have same players in y_train and x_train
players = train_target_stats[['player']]
train_clutch_stats = train_clutch_stats.merge(players, on='player')


In [24]:
# making sure that i have same players in y_train and x_train
x = train_clutch_stats[['player']]
train_target_stats = train_target_stats.merge(x, on='player')

In [29]:
#gotta get the same players in x_test and y_test
d = test_target[['player']]
test_clutch_stats = test_clutch_stats.merge(d, on='player')

In [30]:
#gotta get the same players in x_test and y_test
y = test_clutch_stats[['player']]
test_target = test_target.merge(y, on='player')

In [33]:
x_train = train_clutch_stats.drop(['player','league_all_shot_avg',	'league_2pt_avg',	'league_3pt_avg',	'league_hard2pt_avg',	'league_easy2pt_avg',	'clutch_all_shots',	'clutch_2pt',	'clutch_3pt',	'clutch_hard2',	'clutch_easy2'], axis=1)
y_train = train_target_stats['clutch_all_shots']


In [34]:
x_test = test_clutch_stats.drop(['player','league_all_shot_avg',	'league_2pt_avg',	'league_3pt_avg',	'league_hard2pt_avg',	'league_easy2pt_avg',	'clutch_all_shots',	'clutch_2pt',	'clutch_3pt',	'clutch_hard2',	'clutch_easy2'], axis=1)
y_test = test_target['clutch_all_shots']

In [69]:
model= GradientBoostingClassifier()
#print(model)
fit = model.fit(x_train, y_train)
pred= fit.predict(x_test)


In [70]:
recall_score(y_test,pred)

0.44

In [71]:
f1_score(y_test, pred)

0.4

In [72]:
accuracy_score(y_test, pred)

0.5416666666666666

In [74]:
model = XGBClassifier()
model.fit(x_train, y_train)
#print(model)
y_pred=model.predict(x_test)
print("recall: %.2f%%" % recall_score(y_test, y_pred))
print("f1: %.2f%%" % f1_score(y_test, pred))
print("Accuracy: %.3f%%" % accuracy_score(y_test, y_pred))

recall: 0.44%
f1: 0.40%
Accuracy: 0.528%


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
